In [1]:
%matplotlib inline
import gspread
from gspread.models import Spreadsheet
from gspread_dataframe import set_with_dataframe, get_as_dataframe
from bs4 import BeautifulSoup
import requests
from IPython.display import display, Markdown
import pandas as pd
from urllib.parse import urljoin
from io import StringIO, BytesIO
import re
import numpy as np
import seaborn as sns
import plotly.express as px
from sklearn.preprocessing import StandardScaler

In [2]:
# get google sheet to read and store data
gc = gspread.service_account(
    filename="/Users/grahamflemingthomson/Downloads/grahamflemingthomson-164514-04e450fcd324.json"
)
sh = gc.open_by_key(
    "1OqoSsf5YrsJWRWK1WouiwHeTP_prGVmsP7yURgjQnwo"
)

In [3]:
sh.worksheets()

[<Worksheet '2019_top_players' id:228381128>,
 <Worksheet '2020_top_players' id:167849989>]

In [4]:
top_2020 = get_as_dataframe(sh.worksheets()[1])
top_2020.head()

,rank,player_name,total_points,average_points,week_1_pts,week_2_pts,week_3_pts,week_4_pts,week_5_pts,week_6_pts,...,position,player_path,height,weight,dob,age,experience,height_inches,weight_lbs,experience_years
0,1,"White, Devin TBB LB",188.5,12.567,12.0,17.0,5.0,2.7,2,19.5,...,LB,player?L=46381&P=14168,"6'0""",237lbs,1998-02-17 0:00:00,23.0,3 years,72.0,237.0,3.0
1,2,"Smith, Roquan CHI LB",187.1,12.473,8.0,6.5,9.5,17.5,11.5,13,...,LB,player?L=46381&P=13696,"6'1""",236lbs,1997-04-08 0:00:00,24.0,4 years,73.0,236.0,4.0
2,3,"Watt, T.J. PIT LB",179.5,11.967,12.0,17.0,13.0,B,13.5,3.5,...,LB,player?L=46381&P=13214,"6'4""",252lbs,1994-10-11 0:00:00,26.0,5 years,76.0,252.0,5.0
3,4,"Schobert, Joe JAC LB",162.7,10.847,8.5,8.0,6.5,5,3.5,15.7,...,LB,player?L=46381&P=12768,"6'1""",245lbs,1993-11-06 0:00:00,27.0,6 years,73.0,245.0,6.0
4,5,"Jones, Deion ATL LB",158.3,10.553,9.5,5.4,4.0,9,6,18.7,...,LB,player?L=46381&P=12745,"6'1""",222lbs,1994-11-04 0:00:00,26.0,6 years,73.0,222.0,6.0


In [5]:
pivoted_tp = top_2020.pivot(index="player_name", columns="position", values="total_points")
tp_zscores_within_position = StandardScaler().fit_transform(pivoted_tp)

In [6]:
tp_zscores_within_position_df = pd.DataFrame(
    tp_zscores_within_position, 
    columns=pivoted_tp.columns, 
    index=pivoted_tp.index
).bfill(axis=1)\
.iloc[:, 0]\
.to_frame("z_score")\
.reset_index()

In [7]:
top_2020 = top_2020.merge(tp_zscores_within_position_df, on="player_name", how="left")

In [10]:
px.box(
    top_2020, 
    x="position", 
    y="z_score", 
    color="position",
    points="all", 
    hover_data=["player_name", "total_points", "average_points", "age", "experience_years"]
)